## Classiq's YQuantum 2024 Sparse State-Preparation Challenge

Welcome to the Sparse State-Preparation Challenge, presented by Classiq at YQuantum 2024. This challenge is designed for quantum computing enthusiasts of all levels, aiming to enhance your understanding and spur innovation in sparse state-preparation.

### Challenge Overview
- **Objective**: Develop quantum circuits that efficiently prepare sparse quantum states. These states are characterized by a small number of non-zero amplitudes, reducing quantum resource requirements and enhancing practical execution on near-term quantum hardware.

### Background
- **Importance of State Preparation**: State preparation is a crucial first step for many quantum algorithms. It involves setting a quantum state to represent a specific vector of probabilities or functions. The challenge focuses on sparse states, which are pivotal in applications like quantum linear algebra and quantum machine learning.

### Problem Statement
- **Sparse State-Preparation**: Your task is to create circuits that load a quantum state with probabilities corresponding to a given sparse vector. This challenge highlights the nuances of designing circuits that are both efficient and effective for today's quantum computing limitations.

### Resources and Guidance
- **Classiq's Tools**: Utilize Classiq’s platform, QMOD language, and SDK, which provide robust tools for designing and testing quantum algorithms. Participants are encouraged to base their implementations on "An Efficient Algorithm for Sparse Quantum State Preparation" by Niels Gleinig and Torsten Hoefler.

### Goals and Expectations
- **Output Goal**: Efficiently prepare a quantum state reflecting specified probabilities, ensuring high fidelity and resource efficiency.
- **Example**: For input `{‘00000001’: 0.25, ‘00010001’: 0.5, ‘11001000’: 0.25}`, the output quantum state should correspond to these probabilities.


In [1]:
import classiq
classiq.authenticate()

Your user code: QMBZ-WZDL
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=QMBZ-WZDL


In [2]:
from classiq import *
import numpy as np

## Classical Functions
This section includes utility functions for classical pre-processing in quantum algorithms:

- `dict_to_3d_array(sparse_states)`: Converts a dictionary of sparse states into a 3D array format.
- `custom_filter(func, iterable)`: Filters items in an iterable based on a function.
- `unequal_sets(t, n)`: Determines the best qubit to split a set `t` into subsets with a significant size difference.
- `process_subsets(t, n, dif_qubits, dif_values)`: Processes subsets to determine difference qubits and values.
- `toggle_operations(index, n, x_x, ops1, ops2, s)`: Toggles operations based on the index condition.
- `conditional_toggle(ops1, ops2, n, dif, b, s)`: Conditionally toggles based on a difference.
- `calc_alpha_beta(x_1, x_2)`: Calculates and adjusts alpha and beta values.

In [3]:
def dict_to_3d_array(sparse_states):
    array_3d = []

    for key, value in sparse_states.items():
        point = [int(digit) for digit in key]
        array_3d.append([point, value])

    return array_3d

def custom_filter(func, iterable):
    return [item for item in iterable if func(item)]

def unequal_sets(t, n):

    best_qubit = None
    T_0=[]
    T_1=[]
    current_difference = float('-inf')

    for b in range(n):
        #Filter list based on boolean condition
        T_0 = custom_filter(lambda x, b=b: x[0][b] == 0, t)
        T_1 = custom_filter(lambda x, b=b: x[0][b] == 1, t)

        # Check if both sets are non-empty
        if len(T_0) != 0 and len(T_1) != 0:
            difference = abs(len(T_0) - len(T_1))
            #If new max difference
            if difference > current_difference:
                current_difference = difference
                best_qubit = b
                t_0 = T_0
                t_1 = T_1

    return best_qubit,t_0, t_1

def process_subsets(t, n, dif_qubits, dif_values):
    while len(t) > 1:
        b, T_0, T_1 = unequal_sets(t, n)
        dif_qubits.append(b)
        if len(T_0) < len(T_1):
            t = T_0
            dif_values.append(0)
        else:
            t = T_1
            dif_values.append(1)
    return dif_qubits, dif_values, t

def toggle_operations(index,n, x_x, ops1, ops2,s):
    if x_x[0][index] != 1: #Identical code
            ops1 += [1]
            ops2 += [n-1-index]
            for x in s:
                x[0][index]= int(not x[0][index])

def conditional_toggle(ops1,ops2,n,dif,b,s):
    ops1 += [2]
    sx = [n-1-dif,n-1-b]
    ops2 += [sx]
    for x in s:
        if x[0][dif] == 1:
            x[0][b] = int(not x[0][b])

def calc_alpha_beta(x_1,x_2):
    beta = x_1[1]
    alpha = x_2[1]

    x_2[1] = alpha+beta
    alpha = alpha/x_2[1]
    beta = beta/x_2[1]

    return alpha, beta

def calc_angle(alpha, beta):
    return np.arctan((beta)**.5/(alpha**.5))

## Quantum Functions
This section details quantum operations essential for the algorithm:

- Quantum functions corresponding to the classical ones, allowing operations on quantum states based on classical pre-processing.

In [4]:
#TODO implement unitary_control() using Classiq's built in control()
@qfunc
def unitary_control(qubit: QArray[QBit], contrl: QArray[QBit], target: QParam[int]):
      # works by indexing to the target qbit in the abit array calling X on this qbit
      # and also passing the contrl qbit array as a parameter for the control call
      control(lambda: X(qubit[target]), contrl)

#TODO implement y_rotation using the RY() gate
@qfunc
def y_rotation(theta: QParam[float], reg: QArray[QBit], target: QParam[int]):
      # literally calling the RY function with theat as the angle and indexing to the target qbit in the reg qbit array
      RY(theta, reg[target])

@qfunc
def my_controlled_unitary(q:QArray[QBit], w:QParam[float], ctrl:QArray[QBit], target:QParam[int]) -> None:
      within_apply(compute=lambda: y_rotation(w,q,target), action=lambda: unitary_control(q, ctrl, target))

@qfunc
def my_unitary(q:QArray[QBit], w:QParam[float], target:QParam[int]) -> None:
      within_apply(compute=lambda: y_rotation(w,q,target), action=lambda: X(q[target]))

## Sparse State Prep
Describes the arguments and setup for preparing a sparse quantum state:

- Initial setup includes defining quantum bit requirements and preparing the environment.
- Integration of classical functions to set up and manipulate quantum state preparation based on given sparse states.

### Algorithm 1
- **Purpose**: The primary function of `algorithm_1` is to efficiently prepare a sparse quantum state based on a given set of sparse state descriptions. It utilizes classical pre-processing to determine the optimal sequence of quantum gates.
- **Process**:
  - It begins by identifying the qubits and the operations that will result in the greatest simplification of the quantum state based on the input data.
  - The algorithm iteratively processes subsets of quantum states, toggling qubits and adjusting their probabilities to converge towards the desired sparse state.
  - Each iteration updates operation lists which are used to guide the quantum operations on the actual quantum hardware.
  

In [5]:
def algorithm_1(s,n, ops1, ops2, ops3, ops4, ops5, n9):
    dif_qubits = [] #Where to operate
    dif_values = [] #What operation

    #Preprocessing
    T = s

    dif_qubits, dif_values, t = process_subsets(T, n, dif_qubits, dif_values)

    dif = dif_qubits.pop()
    dif_values.pop()

    x_1 = t[0]
    t_dagger = [x for x in s if all(x[0][q] == v for q, v in zip(dif_qubits, dif_values))]
    t_dagger.remove(x_1)



    dif_qubits, dif_values, t_dagger = process_subsets(t_dagger, n, dif_qubits, dif_values)

    x_2 = t_dagger[0]


    #Storing necessary operations
    toggle_operations(dif,n,x_1,ops1,ops2,s)

    for b in range(n):
        if b != dif and x_1[0][b] != x_2[0][b]:
            conditional_toggle(ops1,ops3,n,dif,b,s)

    for b in dif_qubits:
        toggle_operations(b,n,x_2,ops1,ops2,s)


    alpha, beta = calc_alpha_beta(x_1,x_2)

    ops1 += [3]
    sy = [alpha,beta, dif_qubits, dif]

    if len(dif_qubits) > 0:
        n9 += [len(dif_qubits)]
    else:
        sy.remove(dif_qubits)

    ops4 += [sy]
    s.remove(x_1)

    if len(s) > 1:
        algorithm_1(s,n, ops1, ops2, ops3, ops4, ops5, n9)
    else:
        ops1 += [4]
        ops5 += [x_2[0]]

### Main Function Implementation
- **Purpose**: The `main` function orchestrates the overall setup and execution of the sparse state preparation.
- **Functionality**:
  - It initializes the quantum environment with the necessary number of qubits based on the input state descriptions.
  - The sparse states are first converted into a format suitable for quantum operations using `dict_to_3d_array`.
  - The main function then invokes `algorithm_1` to find the necessary operation to prepare the sparse quantum state.
  - After `algorithm_1` completes, the `main` function applies the quantum operations to finalize the state preparation.
  - It is responsible for managing and applying the computed operations to the quantum system, effectively translating the classical pre-processing results into quantum manipulations.

In [8]:
@qfunc
def main(psi: Output[QArray[QBit]]):
    # Arguments for algorithm1()
    sparse_states = {'000': 0.25, '001': 0.5, '111': 0.25}
    NUM_QUBITS = len(next(iter(sparse_states)))

    ops1 = []  # Stores operations
    ops2 = []
    ops3 = []
    ops4 = []
    ops5 = []
    n9 = []

    # End of arguments
    # Allocate qubits for output
    allocate(NUM_QUBITS, psi)

    sparse_states = dict_to_3d_array(sparse_states)
    if len(sparse_states[0]) > 1:
        # Reference https://htor.inf.ethz.ch/publications/img/quantum_dac.pdf to understand algorithm_2() and its relationship to implementation
        algorithm_1(sparse_states, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)

        # Implementing quantum operations based on ops1-5
        for i in range(len(ops1)):
            op = ops1[i]
            if op == 1:  # Not gate
                qubit_index = ops2[i]
                X(psi[qubit_index])
            elif op == 2:  # CNOT gate
                control_index, target_index = ops3[i]
                CX(psi[control_index], psi[target_index])
            elif op == 3:  # Custom gate
                # Check if ops4 has enough values at index i
                if i < len(ops4) and len(ops4[i]) == 3:
                    # If not, handle it accordingly
                    alpha, beta, qubit = ops4[i]
                    theta = calc_angle(alpha, beta)
                    my_unitary(q=psi, w=theta, target=qubit)
                else:
                    print(f"Invalid ops4 data at index {i}")
                    continue
            # Add more cases if there are additional types of gates

    else:
        for b in range(NUM_QUBITS):
            if sparse_states[0][0][b] == 1:
                X(psi[NUM_QUBITS - 1 - b])

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Invalid ops4 data at index 2
Opening: https://platform.classiq.io/circuit/1b911363-b29c-432f-9c8a-0bf677a5752e?version=0.39.0


In [18]:
@qfunc
def main(psi: Output[QArray[QBit]]):
    # Define the probabilities dictionary for the sparse states
    sparse_states = {'00000001': 0.25, '00010001': 0.5, '11001000': 0.25}

    # Get the number of qubits based on the length of the first key
    NUM_QUBITS = len(next(iter(sparse_states)))
    ops1 = []  # Stores operations
    ops2 = []
    ops3 = []
    ops4 = []
    ops5 = []
    n9 = []
    # Allocate qubits for output
    allocate(NUM_QUBITS, psi)

    # Convert the sparse states dictionary to a 3D array representation
    sparse_states_array = dict_to_3d_array(sparse_states)

    # Check if there are multiple states in the sparse representation
    if len(sparse_states_array[0]) > 1:
        # Apply algorithm 1
        algorithm_1(sparse_states_array, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)

        # Ensure ops1 is a list before iterating over it
        if isinstance(ops1, list):
            # Implement quantum operations based on ops1-5
            for op_params in ops1:
                if isinstance(op_params, tuple):
                    op, *params = op_params
                    if op == 1:  # Not gate
                        qubit_index = params[0]
                        X(psi[qubit_index])
                    elif op == 2:  # CNOT gate
                        control_index, target_index = params
                        CX(psi[control_index], psi[target_index])
                    elif op == 3:  # Custom gate
                        alpha, beta, qubits = params
                        theta = calc_angle(alpha, beta)
                        my_unitary(q=psi, w=theta, target=qubits[0])
                    # Add more cases if there are additional types of gates

    else:
        # If only one state is present, directly apply X gates to set the state
        for b in range(NUM_QUBITS):
            if sparse_states_array[0][0][b] == 1:
                X(psi[NUM_QUBITS - 1 - b])

# Create the model and synthesize the quantum program
model = create_model(main)
qprog = synthesize(model)
show(qprog)

Opening: https://platform.classiq.io/circuit/9e61ffe3-79f9-48bf-a2f4-05a87b794601?version=0.39.0


In [25]:
@qfunc
def main(psi: Output[QArray[QBit]]):
    # Arguments for algorithm1()
    sparse_states = {'000': 0.25, '001': 0.5, '111': 0.25}
    NUM_QUBITS = len(next(iter(sparse_states)))

    ops1 = []  # Stores operations
    ops2 = []
    ops3 = []
    ops4 = []
    ops5 = []
    n9 = []

    # End of arguments
    # Allocate qubits for output
    allocate(NUM_QUBITS, psi)

    sparse_states = dict_to_3d_array(sparse_states)
    if len(sparse_states[0]) > 1:
        # Reference https://htor.inf.ethz.ch/publications/img/quantum_dac.pdf to understand algorithm_2() and its relationship to implementation
        algorithm_1(sparse_states, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)

        # Implementing quantum operations based on ops1-5
        for i in range(len(ops1)):
            op = ops1[i]
            if op == 1:  # Not gate
                qubit_index = ops2[i]
                X(psi[qubit_index])
            elif op == 2:  # CNOT gate
                control_index, target_index = ops3[i]
                CX(psi[control_index], psi[target_index])
            elif op == 3:  # Custom gate
                # Check if ops4 has enough values at index i
                if i < len(ops4) and len(ops4[i]) == 3:
                    # If not, handle it accordingly
                    alpha, beta, qubit = ops4[i]
                    theta = calc_angle(alpha, beta)
                    my_unitary(q=psi, w=theta, target=qubit)
                else:
                    print(f"Invalid ops4 data at index {i}")
                    continue
            # Add more cases for additional types of gates
            elif op == 4:  # Another custom gate
                # Handle the custom gate operation
                pass
            elif op == 5:  # Yet another custom gate
                # Handle another custom gate operation
                pass
            elif op == 6:  # And another custom gate
                # Handle another custom gate operation
                pass
            # Add more cases for additional types of gates as needed
            # elif op == n:  # Nth custom gate
                # Handle the Nth custom gate operation
                # pass

    else:
        for b in range(NUM_QUBITS):
            if sparse_states[0][0][b] == 1:
                X(psi[NUM_QUBITS - 1 - b])

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Invalid ops4 data at index 2
Opening: https://platform.classiq.io/circuit/d25d55e9-34d0-487c-bb08-f203a167101d?version=0.39.0


In [26]:
@qfunc
def main(psi: Output[QArray[QBit]]):
    # Arguments for algorithm1()
    sparse_states = {'000': 0.25, '001': 0.5, '111': 0.25}
    NUM_QUBITS = len(next(iter(sparse_states)))

    ops1 = []  # Stores operations
    ops2 = []
    ops3 = []
    ops4 = []
    ops5 = []
    n9 = []

    # End of arguments
    # Allocate qubits for output
    allocate(NUM_QUBITS, psi)

    sparse_states = dict_to_3d_array(sparse_states)
    if len(sparse_states[0]) > 1:
        # Reference https://htor.inf.ethz.ch/publications/img/quantum_dac.pdf to understand algorithm_2() and its relationship to implementation
        algorithm_1(sparse_states, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)

        # Implementing quantum operations based on ops1-5
        for i in range(len(ops1)):
            op = ops1[i]
            if op == 1:  # Not gate
                qubit_index = ops2[i]
                X(psi[qubit_index])
            elif op == 2:  # CNOT gate
                control_index, target_index = ops3[i]
                CX(psi[control_index], psi[target_index])
            elif op == 3:  # Custom gate
                # Check if ops4 has enough values at index i
                if i < len(ops4) and len(ops4[i]) == 3:
                    # If not, handle it accordingly
                    alpha, beta, qubit = ops4[i]
                    theta = calc_angle(alpha, beta)
                    my_unitary(q=psi, w=theta, target=qubit)
                else:
                    print(f"Invalid ops4 data at index {i}")
                    continue
            elif op == 4:  # Another custom gate
                # Check if ops5 has enough values at index i
                if i < len(ops5) and len(ops5[i]) == 1:
                    # If not, handle it accordingly
                    qubit_index = ops5[i][0]
                    H(psi[qubit_index])
                else:
                    print(f"Invalid ops5 data at index {i}")
                    continue
            # Add more cases for additional types of gates
            # elif op == 5:  # Yet another custom gate
            #     pass
            # elif op == 6:  # And another custom gate
            #     pass
            # Add more cases for additional types of gates as needed
            # elif op == n:  # Nth custom gate
            #     pass

    else:
        for b in range(NUM_QUBITS):
            if sparse_states[0][0][b] == 1:
                X(psi[NUM_QUBITS - 1 - b])

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Invalid ops4 data at index 2
Invalid ops5 data at index 3
Opening: https://platform.classiq.io/circuit/37bb8e02-f085-4222-be2b-39bc82fa481a?version=0.39.0


In [29]:
@qfunc
def main(psi: Output[QArray[QBit]]):
    # Arguments for algorithm1()
    sparse_states = {'000': 0.25, '001': 0.5, '111': 0.25}
    NUM_QUBITS = len(next(iter(sparse_states)))

    ops1 = []  # Stores operations
    ops2 = []
    ops3 = []
    ops4 = []
    ops5 = []
    n9 = []

    # End of arguments
    # Allocate qubits for output
    allocate(NUM_QUBITS, psi)

    sparse_states = dict_to_3d_array(sparse_states)
    if len(sparse_states[0]) > 1:
        # Reference https://htor.inf.ethz.ch/publications/img/quantum_dac.pdf to understand algorithm_2() and its relationship to implementation
        algorithm_1(sparse_states, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)

        # Implementing quantum operations based on ops1-5
        for i in range(len(ops1)):
            op = ops1[i]
            if op == 1:  # Not gate
                if i < len(ops2):
                    qubit_index = ops2[i]
                    X(psi[qubit_index])
                else:
                    print(f"Invalid ops2 data at index {i}")
                    continue
            elif op == 2:  # CNOT gate
                if i < len(ops3):
                    control_index, target_index = ops3[i]
                    CX(psi[control_index], psi[target_index])
                else:
                    print(f"Invalid ops3 data at index {i}")
                    continue
            elif op == 3:  # Custom gate
                # Check if ops4 has enough values at index i
                if i < len(ops4) and len(ops4[i]) == 3:
                    # If not, handle it accordingly
                    alpha, beta, qubit = ops4[i]
                    theta = calc_angle(alpha, beta)
                    my_unitary(q=psi, w=theta, target=qubit)
                else:
                    print(f"Invalid ops4 data at index {i}")
                    continue
            elif op == 4:  # Another custom gate
                # Check if ops5 has enough values at index i
                if i < len(ops5) and len(ops5[i]) == 1:
                    # If not, handle it accordingly
                    qubit_index = ops5[i][0]
                    H(psi[qubit_index])
                else:
                    print(f"Invalid ops5 data at index {i}")
                    continue
            elif op == 5:  # Another custom gate
                # Handle ops5
                pass
            elif op == 6:  # Another custom gate
                # Handle ops6
                pass
            # Add more cases for additional types of gates as needed

    else:
        for b in range(NUM_QUBITS):
            if sparse_states[0][0][b] == 1:
                X(psi[NUM_QUBITS - 1 - b])

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Invalid ops4 data at index 2
Invalid ops5 data at index 3
Opening: https://platform.classiq.io/circuit/017072f9-7e73-4c4c-b519-30dbb823a932?version=0.39.0


## Good Luck!

HINT: If you are getting inconsistent results when running code cells repeatedly, run all cells at once or restart your notebook kernel.